Importing Headers

In [4]:
import csv
import pandas as pd
import numpy as np
import math


Checks whether the age of people is correct as per the bieth year
If (Age of person) > (year when survey was taken) - (year of birth of person) then either the birth year of person has been recorded incorrectly or the age has been recorden incorrectly
In either case we remove that data entry and return the new data frame with index reset

In [2]:
def checkAge(data):
    year_of_survey = 2014
    year_of_birth = data['year_of_birth']
    age = data['Age']
    index = 0 # to keep track of row index
    for i,j in zip(year_of_birth,age):
        if(math.isnan(i) or math.isnan(j)):
            data.drop([index],axis=0,inplace=True) #drop row if age or birth year not recorded
        elif(j > year_of_survey-i):
            data.drop([index],axis = 0,inplace=True) #drop row based on condition mentioned above
        index = index + 1
    data = data.reset_index(drop=True) #reset index of dataframe
    return data

Checks whether the Blood Pressure of the people is within the range of maximum and minimum possible values of blood pressure
If not then the data entry has not benn recorded correctly and hence is removed from the data frame
If the difference between 2 readings of blood pressure of a person is greater than a predefined error then either of the one reading has been recorded incorrectly and thus the corresponding data entry has been removed

In [3]:
def checkBP(data):
    max_sys = 180 #maximum possible value of systolic blood pressure above which a persion has to be hospitalized
    min_sys = 90 #minimun possible value of systolic blood pressure
    max_dia = 120 #maximum possible value of diastolic blood pressure
    min_dia = 60 #minimum possible value of diastolic blood pressure
    max_reading_err = 10 #reading error threshold defined by the user
    data.replace('NA',np.NAN,inplace=True)
    convert_dict = {
        'BP_systolic' : float,
        'BP_systolic_2_reading' : float,
        'BP_Diastolic' : float,
        'BP_Diastolic_2reading' : float
    }
    data = data.astype(convert_dict) #converting data types of columns to float
    s1 = data['BP_systolic']
    s2 = data['BP_systolic_2_reading']
    d1 = data['BP_Diastolic']
    d2 = data['BP_Diastolic_2reading']

    index = 0
    for i,j,k,l in zip(s1,s2,d1,d2):
        if(math.isnan(i) or math.isnan(j) or math.isnan(k) or math.isnan(l)): #missing values will be filled by the column mean later
            index = index + 1
            continue
        elif(abs(i-j)>max_reading_err):
            data.drop([index],axis=0,inplace=True) #remove data entry with error during recording values of systolic BP
        elif(abs(k-l) > max_reading_err):
            data.drop([index],axis=0,inplace=True) #remove data entry with error during recording values of diastolic BP
        elif((i+j)/2 > max_sys or (i+j)/2 < min_sys):
            data.drop([index],axis=0,inplace=True) #remove wrong systolic BP entries 
        elif((k+l)/2 > max_dia or (k+j)/2 < min_dia):
            data.drop([index],axis=0,inplace=True) #remove wrong diastolic BP entries 
        index = index + 1
    data = data.reset_index(drop=True) #reset index
    return data

Checks for invalid dates and months

In [4]:
def checkDayMonth(data):
    convert_dict = {
        'date_of_birth' : float,
        'month_of_birth' : float
    }
    data = data.astype(convert_dict)
    data.drop(data.loc[data['date_of_birth']==0].index, inplace=True) #remove data entry if date of birth is 0
    data.drop(data.loc[data['month_of_birth']==0].index, inplace=True) #remove data entry if month of birth is 0
    data = data.reset_index(drop=True)
    return data

Checks for invalid Pulse rates recorded, i.e. if the pulse rate of a person is not in the range of the maximum and the minimum possible values of pulse rate then the corresponding data entry is removed
If the difference between 2 readings of pulse rate of a person is greater than a predefined error then either of the one reading has been recorded incorrectly and thus the corresponding data entry has been removed

In [5]:
def checkPulseRate(data):
    data = data.replace('NA',np.NAN)
    convert_dict = {
        'Pulse_rate' : float,
        'Pulse_rate_2_reading' : float,
        'Age' : float
    }
    data = data.astype(convert_dict) #changing dataypes of attributes
    prate1 = data['Pulse_rate']
    prate2 = data['Pulse_rate_2_reading']
    age = data['Age']
    #max pulse = 220 - age
    min_pulse = 40
    max_reading_err = 10
    index = 0
    for i,j,k in zip(prate1,prate2,age):
        if(math.isnan(i) or math.isnan(j)): #missing values have been filled later
            index = index + 1
            continue
        elif(abs(i-j) > max_reading_err):
            data = data.drop([index],axis = 0)
        elif((i+j)/2 > (220-k) or (i+j)/2 < min_pulse):
            data = data.drop([index],axis = 0) #remove wrong pulse rate entries
        index = index + 1
    index = 0
    data = data.reset_index(drop=True) #reset Index
    return data

Checks for invalid blood glucose level, i.e. if the blood glucose level of a person is not in the range of the maximum and the minimum possible values of blood glucose level then the corresponding data entry is removed

In [6]:
def checkBloodGlucose(data):
    data = data.replace('NA',np.NAN)
    convert_dict = {
        'fasting_blood_glucose_mg_dl' : float
    }
    data = data.astype(convert_dict) #converting data types
    blood_glucose = data['fasting_blood_glucose_mg_dl']
    max_blood_glucose = 300 #src : wikipedia
    min_blood_glucose = 15

    index = 0
    for i in blood_glucose:
        if(math.isnan(i)): #missing values have been filled later
            index = index + 1
            continue
        elif(i > max_blood_glucose or i < min_blood_glucose):
            data = data.drop([index],axis = 0) #remove wrong blood glucose level entries
        index = index + 1
    index = 0
    data = data.reset_index(drop=True) #reset index
    return data

Checks for invalid haemoglobin level, i.e. if the haemoglobin level of a person is not in the range of the maximum and the minimum possible values of haemoglobin level then the corresponding data entry is removed

In [7]:
def checkHGB(data):
    data = data.replace('NA',np.NAN)
    convert_dict = {
        'Haemoglobin_level' : float
    }
    data = data.astype(convert_dict) #converting data types
    hgb_level = data['Haemoglobin_level']
    max_hgb = 18 #src : wikipedia
    min_hgb = 5

    index = 0
    for i in hgb_level:
        if(math.isnan(i)): #missing values have been filled later
            index = index + 1
            continue
        elif(i > max_hgb or i < min_hgb):
            data = data.drop([index],axis = 0) #remove wrong haemoglobin level entries
        index = index + 1
    index = 0
    data = data.reset_index(drop=True) #reset index
    return data

Driver function
Calls the various functions above after reading the input uncleaned .csv file
Missing data entries have been removed
Attributes having fraction of NA values greater than a predefined user threshold are removed
Missing values have been filled with the column mean

In [2]:
def cleanData(file_name,file_path,output_dir):
    
    fields = []
    
    

    
    itr = 0
    filename = file_path + '/' + file_name
    with open(filename,'r') as data_MP:
        csvreader = csv.reader(data_MP)
        for row in csvreader:
            fields = row
            break

    itr = 0
    filename = file_path + '/' + file_name

    data_frame = pd.read_csv(filename,encoding='utf-8')
    # print(data_frame)
    data_frame = data_frame.replace('NA',np.NAN)
    itr = 0
    threshold = 0.75
    
    useless_fields = []

    for col in data_frame.columns:
        print(str(col) + ' : ' + str(data_frame[col].isna().sum()))
        if(data_frame[col].isna().sum() > threshold*data_frame.shape[0]):
            useless_fields.append(col)
            data_frame = data_frame.drop(columns=col)

    itr = 0

    print("------------------------------------------")
    print("useless fields:")
    for item in useless_fields:
        print(item)
    print("------------------------------------------")

    print("------------------------------------------")
    print("Number of Fieldsss")
    print(len(fields))
    print("Number of USeless Fieldsss")
    print(len(useless_fields))
    print("------------------------------------------")

    print("------------------------------------------")
    print(data_frame)
    print("------------------------------------------")    
    
    
    for col in data_frame.columns:
        data_frame.drop(data_frame.loc[data_frame[col]=='Member - not present'].index, inplace=True)

    print("------------------------------------------")
    print("Rows after 'Member Not Present'")
    print(data_frame.shape[0])
    print("------------------------------------------")

    data_frame = data_frame.reset_index(drop=True)
        
    
    print("------------------------------------------")
    """ Removing duplicates """
    print("Number rows : %d" %(data_frame.shape[0]))
    # print(len(rows))
    dups = data_frame.duplicated()
    print("Number of duplicates = %d" %(dups.sum()))
    data_frame = data_frame.drop_duplicates()
    print("Rows after removing duplicates : %d" %(data_frame.shape[0]))


    print("------------------------------------------")
    """ Removing wrong Age """
    data_frame.drop(data_frame.loc[data_frame['Age']=='NA'].index, inplace=True)
    convert_dict = {
        'Age' : float,
        'year_of_birth' : float
    }
    data_frame = data_frame.astype(convert_dict)
        
    print("Number rows : %d" %(data_frame.shape[0]))
    data_frame = checkAge(data_frame)
    print("Rows after Wrong Age : %d" %(data_frame.shape[0]))

    print("------------------------------------------")
    """ Removing wrong Date and Month of birth"""
    print("Number rows : %d" %(data_frame.shape[0]))
    data_frame = checkDayMonth(data_frame)
    print("Rows after Wrong Date and Month of birth : %d" %(data_frame.shape[0]))

    print("------------------------------------------")
    """ Removing wrong BP """
    print("Number rows : %d" %(data_frame.shape[0]))
    data_frame = checkBP(data_frame)
    print("Rows after Wrong BP : %d" %(data_frame.shape[0]))


    print("------------------------------------------")
    """ Removing wrong Pulse Rate"""
    print("Number rows : %d" %(data_frame.shape[0]))
    data_frame = checkPulseRate(data_frame)
    print("Rows after Wrong Pulse Rate : %d" %(data_frame.shape[0]))


    print("------------------------------------------")
    """ Removing wrong fasting glucose"""
    print("Number rows : %d" %(data_frame.shape[0]))
    data_frame = checkBloodGlucose(data_frame)
    print("Rows after Wrong Blood Glucose : %d" %(data_frame.shape[0]))


    print("------------------------------------------")
    """ Removing wrong Haemoglobin"""
    print("Number rows : %d" %(data_frame.shape[0]))
    data_frame = checkHGB(data_frame)
    print("Rows after Wrong Haemoglobin : %d" %(data_frame.shape[0]))

    data_frame = data_frame.replace('NA',np.NAN)

    data_frame['Haemoglobin_level'].fillna(data_frame['Haemoglobin_level'].mean(),inplace=True)
    data_frame['fasting_blood_glucose_mg_dl'].fillna(data_frame['fasting_blood_glucose_mg_dl'].mean(),inplace=True)
    data_frame['Pulse_rate'].fillna(data_frame['Pulse_rate'].mean(),inplace=True)
    data_frame['Pulse_rate_2_reading'].fillna(data_frame['Pulse_rate_2_reading'].mean(),inplace=True)
    data_frame['BP_systolic'].fillna(data_frame['BP_systolic'].mean(),inplace=True)
    data_frame['BP_systolic_2_reading'].fillna(data_frame['BP_systolic_2_reading'].mean(),inplace=True)
    data_frame['BP_Diastolic'].fillna(data_frame['BP_Diastolic'].mean(),inplace=True)
    data_frame['BP_Diastolic_2reading'].fillna(data_frame['BP_Diastolic_2reading'].mean(),inplace=True)

    convert_dict = {
        'Weight_in_kg' : float,
        'Length_height_cm' : float  
    }
    data_frame = data_frame.astype(convert_dict)

    data_frame['Weight_in_kg'].fillna(data_frame['Weight_in_kg'].mean(),inplace=True)

    data_frame['Length_height_cm'].fillna(data_frame['Length_height_cm'].mean(),inplace=True)

    data_frame = data_frame.fillna('Measured')

    data_frame['length_height_code'].replace('Measured','Height',inplace = True)
    data_frame['Haemoglobin_test'].replace('Measured','YES',inplace = True)
    data_frame['Diabetes_test'].replace('Measured','YES',inplace = True)

    output_file = output_dir + 'cleaned' + file_name 
    data_frame.to_csv(output_file,index=False)


Driver Function to Clean the raw data
Calls the function cleanData defined above
Saves the cleaned files in the cleanedData folder inside the Data folder

In [ ]:
state_list = ['Chhattisgarh','Jharkhand','Odisha','Bihar','MP','UP','Uttarakhand']
file_list = ['Chhattisgarh','Jharkhand','Odisha','Bihar','MadhyaPradesh','UttarPradesh','Uttarakhand']

root_path = './../Data/'
out_dir = './../Data/cleanedData/'

for state,fp in zip(state_list,file_list):
    file_path = root_path + state
    file_name = 'merged' + fp + '.csv'
    cleanData(file_name,file_path,out_dir)